In [1]:
file_path = "C:\\Users\\mutua\\Documents\\Repository\\Repository\\Artificial_Intelligence\\Thesis\\Sequential_Hybrid\\XML100_1144_01.vrp"

In [2]:
def read_cvrp(file_path):
    coords = {}
    demand = {}
    depot = None  # usually node 1 is depot
    with open(file_path) as f:
        lines = f.readlines()
    section = None
    for line in lines:
        line = line.strip()
        print(line)
        if line.startswith("NODE_COORD_SECTION"):
            section = "coord"
            print("here")
            continue
        elif line.startswith("DEMAND_SECTION"):
            section = "demand"
            continue
        elif line.startswith("DEPOT_SECTION"):
            section = "deport"
        elif line.startswith("EOF"):
            break
        if section == "coord":
            parts = line.split()
            if len(parts) >= 3:
                node_id = int(parts[0])
                x = float(parts[1])
                y = float(parts[2])
                coords[node_id] = (x,y)
        elif section == "demand":
            parts = line.split()
            if len(parts) >= 2:
                node_id = int(parts[0])
                q = int(parts[1])
                demand[node_id] = q
        elif section == "deport":
            if line == "-1":
                continue
            try:
                depot = int(line)
            except:
                continue
        
    return depot, coords, demand

In [3]:
deport, coords, demand = read_cvrp(file_path)

NAME : XML100_1144_01
COMMENT : Generated as the XML100 dataset from the CVRPLIB
TYPE : CVRP
DIMENSION : 101
EDGE_WEIGHT_TYPE : EUC_2D
CAPACITY : 783
NODE_COORD_SECTION
here
1    430  41
2    16   749
3    408  727
4    288  143
5    824  940
6    256  931
7    553  208
8    14   95
9    695  603
10   845  972
11   589  247
12   195  939
13   537  282
14   581  227
15   601  294
16   244  822
17   241  869
18   343  865
19   716  945
20   849  553
21   505  847
22   773  97
23   617  560
24   277  119
25   281  461
26   722  225
27   310  991
28   625  655
29   829  616
30   523  497
31   504  676
32   70   91
33   103  362
34   266  487
35   133  897
36   39   862
37   886  533
38   545  722
39   834  944
40   429  593
41   265  988
42   990  145
43   929  223
44   888  341
45   455  93
46   488  453
47   380  813
48   516  142
49   249  747
50   700  338
51   118  498
52   610  817
53   444  323
54   822  556
55   488  366
56   191  33
57   597  913
58   82   919
59   923  150
60   6

In [4]:
from math import sqrt

def create_travel_time(coords):
    travel_time = {}
    for i in coords:
        for j in coords:
            if i != j:
                xi, yi = coords[i]
                xj, yj = coords[j]
                travel_time[(i,j)] = sqrt((xi-xj)**2 + (yi-yj)**2)
    return travel_time

In [5]:
travel_time = create_travel_time(coords)

In [6]:
orders = {}
service_time_default = 5  # e.g., 5 min per delivery
for node, q in demand.items():
    if node == "depot":
        continue
    orders[node] = {
        'a': 0,            # earliest time (no real constraint)
        'b': 1000,         # latest time (large to allow delivery)
        's': service_time_default,
        'q': q
    }

In [7]:
num_vehicles = 5
vehicles = [f'V{i+1}' for i in range(num_vehicles)]
capacities = {v: 783 for v in vehicles}  # from CVRP file

In [8]:
print(deport, coords, demand, travel_time, vehicles, capacities, orders)

1 {1: (430.0, 41.0), 2: (16.0, 749.0), 3: (408.0, 727.0), 4: (288.0, 143.0), 5: (824.0, 940.0), 6: (256.0, 931.0), 7: (553.0, 208.0), 8: (14.0, 95.0), 9: (695.0, 603.0), 10: (845.0, 972.0), 11: (589.0, 247.0), 12: (195.0, 939.0), 13: (537.0, 282.0), 14: (581.0, 227.0), 15: (601.0, 294.0), 16: (244.0, 822.0), 17: (241.0, 869.0), 18: (343.0, 865.0), 19: (716.0, 945.0), 20: (849.0, 553.0), 21: (505.0, 847.0), 22: (773.0, 97.0), 23: (617.0, 560.0), 24: (277.0, 119.0), 25: (281.0, 461.0), 26: (722.0, 225.0), 27: (310.0, 991.0), 28: (625.0, 655.0), 29: (829.0, 616.0), 30: (523.0, 497.0), 31: (504.0, 676.0), 32: (70.0, 91.0), 33: (103.0, 362.0), 34: (266.0, 487.0), 35: (133.0, 897.0), 36: (39.0, 862.0), 37: (886.0, 533.0), 38: (545.0, 722.0), 39: (834.0, 944.0), 40: (429.0, 593.0), 41: (265.0, 988.0), 42: (990.0, 145.0), 43: (929.0, 223.0), 44: (888.0, 341.0), 45: (455.0, 93.0), 46: (488.0, 453.0), 47: (380.0, 813.0), 48: (516.0, 142.0), 49: (249.0, 747.0), 50: (700.0, 338.0), 51: (118.0, 498

In [9]:
"""depot, coords, demand = read_cvrp("XML100_1144_01.vrp")
travel_time = create_travel_time(coords)"""


'depot, coords, demand = read_cvrp("XML100_1144_01.vrp")\ntravel_time = create_travel_time(coords)'

In [15]:
from hybrid import HybridDelivery

In [16]:

hybrid = HybridDelivery(
    orders=orders,
    vehicles=vehicles,
    travel_time=travel_time,
    capacities=capacities,
    service_time={o: orders[o]['s'] for o in orders}
)


In [17]:

history, final_state, completed_plan = hybrid.run(max_steps=1000)


Step 0 | Pending: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101}
Disruption detected! MDP chooses action: continue
Step 1 | Pending: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101}
Disruption detected! MDP chooses action: continue
Step 2 | Pending: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15

In [18]:

# Example output
print("Completed Plan per vehicle:")
for v, route in history:
    print(f"{v}: {route}")

Completed Plan per vehicle:
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101}: 0
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101}: 0
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45

In [19]:
print(final_state)

{'vehicle_positions': {'V1': {'location': 6, 'time': 0, 'remaining_travel': 680.7529165241498, 'next_order': 11, 'service_remaining': 5}, 'V2': {'location': 2, 'time': 0, 'remaining_travel': 590.2663576467218, 'next_order': 7, 'service_remaining': 5}, 'V3': {'location': 3, 'time': 0, 'remaining_travel': 438.7549932695987, 'next_order': 8, 'service_remaining': 5}, 'V4': {'location': 9, 'time': 0, 'remaining_travel': 195.90202341041714, 'next_order': 14, 'service_remaining': 5}, 'V5': {'location': 5, 'time': 0, 'remaining_travel': 27.275318418009277, 'next_order': 10, 'service_remaining': 5}}, 'pending_orders': {7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101}, 'done